In [3]:
# ============================================================
# run_blogfeedback_ppci_mean_updated.py
#
# BlogFeedback conditional mean simulation (UPDATED to match census_income logic)
#
# Key updates vs old BlogFeedback version:
#   1) Gaussian kernel -> Matérn 5/2 kernel (via conditional_mean_functions.py)
#   2) pilot: LOO-CV on log(ell) grid (reuse LOGO-CV by giving unique groups)
#   3) lambda grid scales like O(1/n_label)
#   4) do NOT treat N_t as an independent budget:
#        N_unlab_total = N_t_full + N_use, and MC samples only N_use each replicate
#   5) Save sigma^2 items (MC mean/std):
#        PPCI:    sigma2_Y_minus_A, sigma2_A
#        LabelOnly: sigma2_Y
#
# NEW CHANGE (requested):
#   - ell tuning: compute pilot-based ell0_hat = median(||X_pilot - x0||)
#     then run LOO-CV on LOGELL_grid_local centered at ell0_hat
#     (no longer uses external LOGELL_grid from main loop)
# ============================================================

import numpy as np
import cupy as cp
import pandas as pd
import os
import sys
import csv

from ppi_py import ppi_mean_ci, ppi_mean_pointestimate, crossppi_mean_ci

# GPU backend
xp = cp
np.set_printoptions(precision=4, suppress=True)

# 兼容：脚本里用 __file__，notebook 里用 os.getcwd()
try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from conditional_mean_functions import (
    select_logell_by_pilot_logo_cv,
    nw_point_mean_ci_matern52,
    WeightAtX0,
    ppci_conditional_mean,
    ppci_conditional_mean_label_only,
)

# -------------------------
# helper: write CSV
# -------------------------
def write_results_to_csv(results_list, csv_path):
    if len(results_list) == 0:
        raise ValueError("results_list is empty; nothing to write.")
    fieldnames = list(results_list[0].keys())
    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in results_list:
            writer.writerow(row)
    print(f"\n[Saved] {csv_path}")


# -------------------------
# core: simulate one x0 (matches census_income style)
# -------------------------
def simulate_one_x0_blogfeedback(
    Y_total,
    Yhat_total,
    X_total,            # standardized already
    x0_scaled,          # standardized already
    seed,
    B,
    n_label,
    N_unlab_total,      # = N_t_full + N_use
    N_t_full,
    lam_grid,
    LOGELL_grid,        # kept for signature compatibility (ignored below)
    n_pilot,
    alpha,
    z_alpha,
    theta0_mode="smooth",
):
    rng = np.random.default_rng(seed)

    X_total = np.asarray(X_total, dtype=float)
    Y_total = np.asarray(Y_total, dtype=float)
    Yhat_total = np.asarray(Yhat_total, dtype=float)
    x0_scaled = np.asarray(x0_scaled, dtype=float).ravel()

    # ----- enforce N_unlab_total split -----
    if N_unlab_total < N_t_full:
        raise ValueError("N_unlab_total must be >= N_t_full.")
    N_use = int(N_unlab_total - N_t_full)
    if N_use <= 0:
        raise ValueError("Need N_use = N_unlab_total - N_t_full > 0 for PPCI.")

    # ----- exclude exact duplicates of x0 (same as income script) -----
    mask_x0 = np.all(X_total == x0_scaled.reshape(1, -1), axis=1)
    n_x0 = int(mask_x0.sum())
    print(f"[x0] exact duplicate observations (excluded): {n_x0}")

    theta0_obs = float(np.mean(Y_total[mask_x0])) if np.any(mask_x0) else np.nan

    # theta0_smooth via NW on full pool (oracle ell = median distance)
    dists = np.linalg.norm(X_total - x0_scaled.reshape(1, -1), axis=1)
    ell_oracle = float(np.median(dists))

    X_full_gpu = cp.asarray(X_total)
    Y_full_gpu = cp.asarray(Y_total)
    theta0_s, _, _ = nw_point_mean_ci_matern52(
        X_l=X_full_gpu,
        Y_l=Y_full_gpu,
        x0=cp.asarray(x0_scaled),
        ell=ell_oracle,
        z_alpha=z_alpha,
    )
    theta0_smooth = float(theta0_s)

    if theta0_mode == "obs":
        if np.isnan(theta0_obs):
            raise ValueError("theta0_mode='obs' but x0 has no exact obs.")
        theta0 = theta0_obs
        theta0_source = "obs"
    elif theta0_mode == "smooth":
        theta0 = theta0_smooth
        theta0_source = "smooth"
    elif theta0_mode == "mix":
        if not np.isnan(theta0_obs):
            theta0 = theta0_obs
            theta0_source = "obs"
        else:
            theta0 = theta0_smooth
            theta0_source = "smooth"
    else:
        raise ValueError("theta0_mode must be 'obs'/'smooth'/'mix'.")

    # used pool (exclude duplicates)
    X_used = X_total[~mask_x0]
    Y_used = Y_total[~mask_x0]
    Yhat_used = Yhat_total[~mask_x0]
    N_used = int(Y_used.size)

    # ----- split requirements (pilot + aux + eval_pool) -----
    min_required = n_pilot + N_t_full + (n_label + N_use)
    if N_used < min_required:
        raise ValueError(
            f"N_used={N_used} too small for pilot({n_pilot}) + aux({N_t_full}) "
            f"+ labeled({n_label}) + unlab_use({N_use})."
        )

    # GPU arrays
    X_used_gpu = cp.asarray(X_used)
    Y_used_gpu = cp.asarray(Y_used)
    Yhat_used_gpu = cp.asarray(Yhat_used)

    # global split: pilot / aux / eval_pool
    perm_all = rng.permutation(N_used)
    idx_pilot = perm_all[:n_pilot]
    idx_aux = perm_all[n_pilot : n_pilot + N_t_full]
    idx_eval_pool = perm_all[n_pilot + N_t_full :]

    # ----- pilot ell tuning: compute ell0_hat then LOO on local grid -----
    X_pilot_gpu = X_used_gpu[idx_pilot]
    Y_pilot_gpu = Y_used_gpu[idx_pilot]
    loo_groups = np.arange(n_pilot, dtype=int)

    # (1) pilot-based ell0_hat = median(||X_pilot - x0||)
    x0_gpu = cp.asarray(x0_scaled)
    d_pilot = cp.linalg.norm(X_pilot_gpu - x0_gpu[None, :], axis=1)
    ell0_hat = float(cp.asnumpy(cp.median(d_pilot)))

    # (2) build LOGELL_grid_local around ell0_hat, then run LOO-CV
    #     You can adjust c_min/c_max if you want wider/narrower search.
    c_min, c_max = 0.925, 1.075
    ell_low = max(1e-8, c_min * ell0_hat)
    ell_high = c_max * ell0_hat
    LOGELL_grid_local = np.linspace(np.log(ell_low), np.log(ell_high), 40)

    ell_star, pilot_table = select_logell_by_pilot_logo_cv(
        LOGELL_grid=LOGELL_grid_local,
        X_pilot=X_pilot_gpu,
        Y_pilot=Y_pilot_gpu,
        group_labels=loo_groups,
    )

    # ----- build f_hat + lambda_hat (x0-specific) -----
    X_aux = X_used_gpu[idx_aux]
    locw = WeightAtX0(X_aux=X_aux, x0=x0_scaled, ell=ell_star)
    lam_hat, alpha_hat, lam_grid_sorted = locw.select_lambda_lcurve(
        lam_grid=lam_grid, normalize=False, make_plot=False
    )
    f_hat = locw.make_f_hat(alpha_hat)

    # ----- MC containers -----
    thetas_ppci, covered_ppci, widths_ppci = [], [], []
    sigma2_YmA_list, sigma2_A_list = [], []

    thetas_lo, covered_lo, widths_lo = [], [], []
    sigma2_Y_list = [],

    thetas_nw, covered_nw, widths_nw = [], [], []   # optional baseline

    thetas_ppi, covered_ppi, widths_ppi = [], [], []
    covered_crossppi, widths_crossppi = [], []

    # ----- MC loop -----
    sigma2_Y_list = []
    for _ in range(B):
        perm_eval = rng.permutation(idx_eval_pool)
        idx_l = perm_eval[:n_label]
        idx_u = perm_eval[n_label : n_label + N_use]

        X_l = X_used_gpu[idx_l]
        Y_l = Y_used_gpu[idx_l]
        A_l = Yhat_used_gpu[idx_l]

        X_u = X_used_gpu[idx_u]
        A_u = Yhat_used_gpu[idx_u]

        # PPCI
        th, se, (lo, up), ex = ppci_conditional_mean(
            X_l=X_l, Y_l=Y_l, X_u=X_u,
            f_hat=f_hat, A_l=A_l, A_u=A_u,
            z_alpha=z_alpha, return_extras=True
        )
        thetas_ppci.append(th)
        covered_ppci.append(float(lo <= theta0 <= up))
        widths_ppci.append(float(up - lo))
        sigma2_YmA_list.append(ex["sigma2_Y_minus_A"])
        sigma2_A_list.append(ex["sigma2_A"])

        # Label-only
        th2, se2, (lo2, up2), ex2 = ppci_conditional_mean_label_only(
            X_l=X_l, Y_l=Y_l, f_hat=f_hat,
            z_alpha=z_alpha, return_extras=True
        )
        thetas_lo.append(th2)
        covered_lo.append(float(lo2 <= theta0 <= up2))
        widths_lo.append(float(up2 - lo2))
        sigma2_Y_list.append(ex2["sigma2_Y"])

        # NW baseline (Matérn NW on labeled, using ell_star)
        thn, sen, (lon, upn) = nw_point_mean_ci_matern52(
            X_l=X_l, Y_l=Y_l, x0=cp.asarray(x0_scaled),
            ell=ell_star, z_alpha=z_alpha
        )
        thetas_nw.append(float(thn))
        covered_nw.append(float(lon <= theta0 <= upn))
        widths_nw.append(float(upn - lon))

        # PPI / crossPPI (global, CPU)
        Y_l_cpu = cp.asnumpy(Y_l).ravel()
        A_l_cpu = cp.asnumpy(A_l).ravel()
        A_u_cpu = cp.asnumpy(A_u).ravel()

        ppi_lo, ppi_up = ppi_mean_ci(Y_l_cpu, A_l_cpu, A_u_cpu, alpha=alpha)
        ppi_th = ppi_mean_pointestimate(Y_l_cpu, A_l_cpu, A_u_cpu)
        thetas_ppi.append(float(ppi_th))
        covered_ppi.append(float(ppi_lo <= theta0 <= ppi_up))
        widths_ppi.append(float(ppi_up - ppi_lo))

        A_l_2d = A_l_cpu.reshape(-1, 1)
        A_u_2d = A_u_cpu.reshape(-1, 1)
        cross_lo, cross_up = crossppi_mean_ci(Y_l_cpu, A_l_2d, A_u_2d, alpha=alpha)
        covered_crossppi.append(float(cross_lo <= theta0 <= cross_up))
        widths_crossppi.append(float(cross_up - cross_lo))

    # ----- summarize -----
    thetas_ppci = np.asarray(thetas_ppci, float)
    covered_ppci = np.asarray(covered_ppci, float)
    widths_ppci = np.asarray(widths_ppci, float)
    sigma2_YmA = np.asarray(sigma2_YmA_list, float)
    sigma2_A = np.asarray(sigma2_A_list, float)

    thetas_lo = np.asarray(thetas_lo, float)
    covered_lo = np.asarray(covered_lo, float)
    widths_lo = np.asarray(widths_lo, float)
    sigma2_Y = np.asarray(sigma2_Y_list, float)

    thetas_nw = np.asarray(thetas_nw, float)
    covered_nw = np.asarray(covered_nw, float)
    widths_nw = np.asarray(widths_nw, float)

    thetas_ppi = np.asarray(thetas_ppi, float)
    covered_ppi = np.asarray(covered_ppi, float)
    widths_ppi = np.asarray(widths_ppi, float)

    covered_crossppi = np.asarray(covered_crossppi, float)
    widths_crossppi = np.asarray(widths_crossppi, float)

    out = {
        # x0 / oracle truth
        "x0_index": -1,  # 外层填
        "theta0_mode": theta0_mode,
        "theta0_source": theta0_source,
        "theta0": float(theta0),
        "theta0_obs": float(theta0_obs) if not np.isnan(theta0_obs) else np.nan,
        "theta0_smooth": float(theta0_smooth),
        "ell_oracle": float(ell_oracle),

        # tuning
        "ell0_hat": float(ell0_hat),
        "ell_low": float(ell_low),
        "ell_high": float(ell_high),
        "ell_star": float(ell_star),
        "lambda_hat": float(lam_hat),

        # budgets
        "B": int(B),
        "n_label": int(n_label),
        "n_pilot": int(n_pilot),
        "N_unlab_total": int(N_unlab_total),
        "N_t_full": int(N_t_full),
        "N_use": int(N_use),

        # PPCI metrics
        "PPCI_theta_mean": float(np.mean(thetas_ppci)),
        "PPCI_theta_rmse": float(np.sqrt(np.mean((thetas_ppci - theta0) ** 2))),
        "PPCI_coverage": float(np.mean(covered_ppci)),
        "PPCI_avg_ci_width": float(np.mean(widths_ppci)),
        "PPCI_sigma2_Y_minus_A_mean": float(np.mean(sigma2_YmA)),
        "PPCI_sigma2_Y_minus_A_std": float(np.std(sigma2_YmA, ddof=1)) if B > 1 else 0.0,
        "PPCI_sigma2_A_mean": float(np.mean(sigma2_A)),
        "PPCI_sigma2_A_std": float(np.std(sigma2_A, ddof=1)) if B > 1 else 0.0,

        # Label-only metrics
        "LO_theta_mean": float(np.mean(thetas_lo)),
        "LO_theta_rmse": float(np.sqrt(np.mean((thetas_lo - theta0) ** 2))),
        "LO_coverage": float(np.mean(covered_lo)),
        "LO_avg_ci_width": float(np.mean(widths_lo)),
        "LO_sigma2_Y_mean": float(np.mean(sigma2_Y)),
        "LO_sigma2_Y_std": float(np.std(sigma2_Y, ddof=1)) if B > 1 else 0.0,

        # NW baseline (optional)
        "NW_theta_mean": float(np.mean(thetas_nw)),
        "NW_theta_rmse": float(np.sqrt(np.mean((thetas_nw - theta0) ** 2))),
        "NW_coverage": float(np.mean(covered_nw)),
        "NW_avg_ci_width": float(np.mean(widths_nw)),

        # PPI metrics
        "PPI_theta_mean": float(np.mean(thetas_ppi)),
        "PPI_theta_rmse": float(np.sqrt(np.mean((thetas_ppi - theta0) ** 2))),
        "PPI_coverage": float(np.mean(covered_ppi)),
        "PPI_avg_ci_width": float(np.mean(widths_ppi)),

        # crossPPI metrics
        "crossPPI_coverage": float(np.mean(covered_crossppi)),
        "crossPPI_avg_ci_width": float(np.mean(widths_crossppi)),
    }

    # also store x0 coordinates (wide columns are annoying; store as string)
    out["x0_vec"] = np.array(x0_scaled, dtype=float).tolist()
    return out


# ============================================================
# entry point
# ============================================================
if __name__ == "__main__":
    # choose GPU
    cp.cuda.Device(0).use()

    # --------------------------------------------------------
    # 1) Load BlogFeedback PPCI data
    # --------------------------------------------------------
    ppci_csv = "./blogfeedback_ppci.csv"
    x0_csv = "./blogfeedback_ppci_x0.csv"

    df_ppci = pd.read_csv(ppci_csv)
    df_x0 = pd.read_csv(x0_csv)

    x_cols = [c for c in df_ppci.columns if c.startswith("x")]
    X_total = df_ppci[x_cols].to_numpy(dtype=np.float32)
    Y_total = df_ppci["logy"].to_numpy(dtype=np.float32)
    Yhat_total = df_ppci["logyhat"].to_numpy(dtype=np.float32)

    n_total, d = X_total.shape
    print(f"Loaded BlogFeedback PPCI data: n_total={n_total}, d={d}")

    x0_list = [row.to_numpy(dtype=np.float32) for _, row in df_x0[x_cols].iterrows()]
    print(f"Loaded {len(x0_list)} x0 points from {x0_csv}")

    # --------------------------------------------------------
    # 2) Hyperparams (UPDATED per your rules)
    # --------------------------------------------------------
    seed = 2025
    B = 500
    n_label = 300

    # IMPORTANT: unlabeled total budget includes localization auxiliary set
    N_t_full = min(7000, n_total - 1)       # localization aux pool size
    N_use = 3000                             # each MC replicate uses this many unlabeled eval samples
    N_unlab_total = N_t_full + N_use        # <-- the change you asked for

    n_pilot = 300
    alpha = 0.05
    z_alpha = 1.96

    # lambda grid: O(1/n_label)
    lam_grid = np.logspace(np.log10(0.05 / n_label), np.log10(10.0 / n_label), 50)

    # dummy LOGELL_grid (ignored; kept to avoid touching other code paths)
    LOGELL_grid_dummy = np.linspace(np.log(10.0), np.log(30.0), 40)

    # --------------------------------------------------------
    # 3) Loop x0 and run (income-style)
    # --------------------------------------------------------
    all_rows = []
    for k, x0 in enumerate(x0_list, 1):
        print("\n" + "-" * 90)
        print(f"[x0 #{k}/{len(x0_list)}] running...")

        out = simulate_one_x0_blogfeedback(
            Y_total=Y_total,
            Yhat_total=Yhat_total,
            X_total=X_total,
            x0_scaled=x0,
            seed=seed,
            B=B,
            n_label=n_label,
            N_unlab_total=N_unlab_total,
            N_t_full=N_t_full,
            lam_grid=lam_grid,
            LOGELL_grid=LOGELL_grid_dummy,   # ignored inside
            n_pilot=n_pilot,
            alpha=alpha,
            z_alpha=z_alpha,
            theta0_mode="smooth",
        )
        out["x0_index"] = int(k - 1)
        all_rows.append(out)

        # ---- print (same spirit as census_income) ----
        print(
            f"theta0={out['theta0']:.6f}  "
            f"ell_oracle={out['ell_oracle']:.4f}  "
            f"ell0_hat={out['ell0_hat']:.4f}  "
            f"ell_star={out['ell_star']:.4f}  "
            f"lambda_hat={out['lambda_hat']:.6g}"
        )

        print(
            "PPCI : "
            f"rmse={out['PPCI_theta_rmse']:.6f}  "
            f"cov={out['PPCI_coverage']:.3f}  width={out['PPCI_avg_ci_width']:.6f}  "
            f"s2_YmA={out['PPCI_sigma2_Y_minus_A_mean']:.6g}  "
            f"s2_A={out['PPCI_sigma2_A_mean']:.6g}"
        )
        print(
            "LO   : "
            f"rmse={out['LO_theta_rmse']:.6f}  "
            f"cov={out['LO_coverage']:.3f}  width={out['LO_avg_ci_width']:.6f}  "
            f"s2_Y={out['LO_sigma2_Y_mean']:.6g}"
        )
        print(
            "NW   : "
            f"rmse={out['NW_theta_rmse']:.6f}  "
            f"cov={out['NW_coverage']:.3f}  width={out['NW_avg_ci_width']:.6f}"
        )
        print(
            "PPI  : "
            f"rmse={out['PPI_theta_rmse']:.6f}  "
            f"cov={out['PPI_coverage']:.3f}  width={out['PPI_avg_ci_width']:.6f}"
        )
        print(
            "cPPI : "
            f"cov={out['crossPPI_coverage']:.3f}  width={out['crossPPI_avg_ci_width']:.6f}"
        )

    # --------------------------------------------------------
    # 4) Save CSV
    # --------------------------------------------------------
    save_dir = os.path.join(current_dir, "results")
    os.makedirs(save_dir, exist_ok=True)
    csv_path = os.path.join(save_dir, "blogfeedback_ppci_all_x0_updated.csv")
    write_results_to_csv(all_rows, csv_path)


Loaded BlogFeedback PPCI data: n_total=14517, d=280
Loaded 50 x0 points from ./blogfeedback_ppci_x0.csv

------------------------------------------------------------------------------------------
[x0 #1/50] running...
[x0] exact duplicate observations (excluded): 0
theta0=0.514891  ell_oracle=17.0527  ell0_hat=16.7480  ell_star=15.4919  lambda_hat=0.00427211
PPCI : rmse=0.037209  cov=0.948  width=0.148147  s2_YmA=0.0688562  s2_A=0.0979973
LO   : rmse=0.055783  cov=0.934  width=0.215107  s2_Y=0.166187
NW   : rmse=0.055855  cov=0.930  width=0.214582
PPI  : rmse=0.109027  cov=0.286  width=0.162537
cPPI : cov=0.950  width=0.337608

------------------------------------------------------------------------------------------
[x0 #2/50] running...
[x0] exact duplicate observations (excluded): 0
theta0=0.541119  ell_oracle=15.1575  ell0_hat=15.1816  ell_star=14.0430  lambda_hat=0.00475995
PPCI : rmse=0.039537  cov=0.928  width=0.153389  s2_YmA=0.0711048  s2_A=0.10524
LO   : rmse=0.059016  cov=0.